<a href="https://colab.research.google.com/github/donyeun/AMLSII_19-20_SN18154195/blob/master/AMLS_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Libraries and Variables

In [0]:
import pandas as pd
import os
import torch
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
import csv

In [0]:
cfg = {
       'paths': {
           'train_folder_task_a': '/content/drive/My Drive/public/AMLSII_19-20_SN18154195/dataset/training/2017_English_final/GOLD/Subtask_A',
           'train_additional_dataset_filenames': ['livejournal-2014test-A.tsv', 'sms-2013test-A.tsv'],
           'test_file_task_a': '/content/drive/My Drive/public/AMLSII_19-20_SN18154195/dataset/testing/SemEval2017-task4-test/SemEval2017-task4-test.subtask-A.english.txt',

           'train_folder_task_b': '/content/drive/My Drive/public/AMLSII_19-20_SN18154195/dataset/training/2017_English_final/GOLD/Subtasks_BD',
           'test_file_task_b': '/content/drive/My Drive/public/AMLSII_19-20_SN18154195/dataset/testing/SemEval2017-task4-test/SemEval2017-task4-test.subtask-BD.english.txt',
       },
       'task_a': {
           'use_additional_dataset': False,
           'dataset': {
               'clean' : {
                   'training_filepath': '/content/drive/My Drive/public/AMLSII_19-20_SN18154195/dataset/clean/task_a_training.csv'
               }
           }
       },
       'preprocessing': {
           'tokenization': {
               'remove_twitter_handle': True,
               'preserve_case': False
           }
       },
}

# Load Dataset and Data Preprocessing

## Preprocess Additional Dataset
In task A, there are some additional datasets that were given from the competition apart from the standard twitter corpus, which is sms and livejournal corpus. We can use both of these additional datasets, after we preprocess the formatting so that it matches the rest of the twitter datasets.

In [18]:
task_a_train_additional_dfs = {}

for filename in cfg['paths']['train_additional_dataset_filenames']:
  # read additional corpora
  task_a_train_additional_dfs[filename] = pd.read_csv(os.path.join(cfg['paths']['train_folder_task_a'], filename), sep='\t', header=None, quoting=csv.QUOTE_NONE)
  
  if filename == 'livejournal-2014test-A.tsv':
    # remove the 0-th column from livejournal corpora, as it is unnecessary
    task_a_train_additional_dfs[filename] = task_a_train_additional_dfs[filename].drop(columns=[0])
  elif filename == 'sms-2013test-A.tsv':
    # remove the 1st column from sms corpora, as it is unnecessary
    task_a_train_additional_dfs[filename] = task_a_train_additional_dfs[filename].drop(columns=[1])
  
  # reset the column index to make it incremental
  task_a_train_additional_dfs[filename].columns = range(task_a_train_additional_dfs[filename].shape[1])
  
  print(task_a_train_additional_dfs[filename])

             0         1                                                  2
0     LJ111111  negative  I know I missed something here , but what does...
1     LJ111113   neutral  What do you think of Beside Ourselves as a tit...
2     LJ111114  positive                    :D I intend to be one someday .
3     LJ111117  negative  LLLINKKK LLLINKKK IIIMAGEEELLLINKKK The choice...
4     LJ111119   neutral                     LLLINKKK Some more mountains .
...        ...       ...                                                ...
1137  LJ113616  positive                     Maybe it was - his - fantasy ?
1138  LJ113618  negative  It was ok , but they always just seem so nervo...
1139  LJ113621  positive  It is streamable from YepRoc -- matter of fact...
1140  LJ113623  positive  comment telling me who you are , or how you fo...
1141  LJ113625   neutral  im on myspace ... ill try and find you and add...

[1142 rows x 3 columns]
          0         1                                          

## Make Datasets As Pandas' DataFrames and Cleaning Dataset



In [0]:
def append_txt_files_as_one_dataframe(folderpath, filename_keywords_list, additional_dataset_dfs=None):
  dataset_per_file_dfs = {}
  dataset_df = pd.DataFrame()
  filenames = os.listdir(folderpath)
  
  # open txt files (in tsv formatting)
  for filename in filenames:
    # if the filename contains a keyword in the filename_keywords_list, then open the txt file
    # this is to avoid opening unnecessary txt such as readme.txt file.
    if any(keyword in filename for keyword in filename_keywords_list):
      dataset_per_file_dfs[filename] = pd.read_csv(os.path.join(folderpath, filename), sep="\t", header=None, quoting=csv.QUOTE_NONE)
      print(dataset_per_file_dfs[filename].shape, filename)

  # combine the original dataset with additional dataset (if any)
  if additional_dataset_dfs is not None:
    for key, value in additional_dataset_dfs.items():
      dataset_per_file_dfs[key] = value

  # append all the files as one dataframe
  for key, value in dataset_per_file_dfs.items():
    # print(key, '\t', i, '\t', dataset_per_file_df[key].shape[0])
    dataset_df = dataset_df.append(dataset_per_file_dfs[key], ignore_index=True)
  return dataset_df

In [0]:
def clean_dataframe_format(df, new_column_name_list, drop_column_list=[]):
  # drop unnecessary column
  df = df.drop(columns=drop_column_list)
  
  # rename column
  df.columns = new_column_name_list

  # remove row in dataframe if the 'text' or 'sentiment' column value is missing
  df = df.dropna(subset=['sentiment', 'text'], how='any').reset_index(drop=True)
  
  # remove row if the sentiment is not 'positive', 'negative' or 'neutral'
  # this happens in the dataset, for example, there are some rows
  # where its sentiments are 'off topic'
  valid_sentiments = ['positive', 'negative', 'neutral']
  df = df[df['sentiment'].isin(valid_sentiments)].reset_index(drop=True)
  return df

In [21]:
# read dataset files and append it as one pandas dataframe
if cfg['task_a']['use_additional_dataset']:
  task_a_train_df = append_txt_files_as_one_dataframe(cfg['paths']['train_folder_task_a'], ['twitter'], task_a_train_additional_dfs)
else:
  task_a_train_df = append_txt_files_as_one_dataframe(cfg['paths']['train_folder_task_a'], ['twitter'])

task_a_train_df = clean_dataframe_format(task_a_train_df, ['id', 'sentiment', 'text'], drop_column_list=[3])

# # save dataset as csv file
# with pd.option_context('display.max_columns', None):  # more options can be specified also
#     print(task_a_train_df)

# print(task_a_train_df.shape)

(2000, 3) twitter-2016devtest-A.txt
(1999, 3) twitter-2016dev-A.txt
(6000, 3) twitter-2016train-A.txt
(1654, 3) twitter-2013dev-A.txt
(3547, 3) twitter-2013test-A.txt
(9684, 3) twitter-2013train-A.txt
(1853, 3) twitter-2014test-A.txt
(2390, 3) twitter-2015test-A.txt
(489, 3) twitter-2015train-A.txt
(86, 3) twitter-2014sarcasm-A.txt
(20633, 4) twitter-2016test-A.txt


In [29]:
task_b_train_df = append_txt_files_as_one_dataframe(cfg['paths']['train_folder_task_b'], ['twitter'])
task_b_train_df = clean_dataframe_format(task_b_train_df, ['id', 'topic','sentiment', 'text'], drop_column_list=[4])
task_b_train_df.info()

(4346, 4) twitter-2016train-BD.txt
(1325, 4) twitter-2016dev-BD.txt
(1417, 4) twitter-2016devtest-BD.txt
(489, 4) twitter-2015train-BD.txt
(10552, 5) twitter-2016test-BD.txt
(2383, 5) twitter-2015testBD.txt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20508 entries, 0 to 20507
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         20508 non-null  object
 1   topic      20508 non-null  object
 2   sentiment  20508 non-null  object
 3   text       20508 non-null  object
dtypes: object(4)
memory usage: 641.0+ KB


In [23]:
print(task_b_train_df[
                (task_b_train_df['sentiment'] != 'negative') &
                (task_b_train_df['sentiment'] != 'positive') &
                (task_b_train_df['sentiment'] != 'neutral')
                ]['sentiment'])

# task_b_train_df

# task_b_train_df[task_b_train_df['text'].str.len()< 40]

Series([], Name: sentiment, dtype: object)


In [24]:
task_a_test_df = pd.read_csv(cfg['paths']['test_file_task_a'], sep='\t', header=None, quoting=csv.QUOTE_NONE)
task_a_test_df = clean_dataframe_format(task_a_test_df, ['id', 'sentiment', 'text'])
task_a_test_df

,id,sentiment,text
0,801989080477154944,neutral,#ArianaGrande Ari By Ariana Grande 80% Full ht...
1,801989272341453952,positive,Ariana Grande KIIS FM Yours Truly CD listening...
2,801990978424962944,positive,Ariana Grande White House Easter Egg Roll in W...
3,801996232553963008,positive,#CD #Musics Ariana Grande Sweet Like Candy 3.4...
4,801998343442407040,neutral,SIDE TO SIDE 😘 @arianagrande #sidetoside #aria...
...,...,...,...
12279,805699615781625856,positive,@dansen17 update: Zac Efron kissing a puppy ht...
12280,805701709356003328,neutral,#zac efron sex pic skins michelle sex https://...
12281,805701818357579776,neutral,First Look at Neighbors 2 with Zac Efron Shirt...
12282,805703557081075712,neutral,zac efron poses nude #lovely libra porn https:...


In [25]:
task_b_test_df = pd.read_csv(cfg['paths']['test_file_task_b'], sep='\t', header=None, quoting=csv.QUOTE_NONE)
task_b_test_df = clean_dataframe_format(task_b_test_df, ['id', 'topic', 'sentiment', 'text'])
task_b_test_df

,id,topic,sentiment,text
0,801989272341453952,#ArianaGrande,positive,Ariana Grande KIIS FM Yours Truly CD listening...
1,801990978424962944,#ArianaGrande,positive,Ariana Grande White House Easter Egg Roll in W...
2,801996232553963008,#ArianaGrande,positive,#CD #Musics Ariana Grande Sweet Like Candy 3.4...
3,801998343442407040,#ArianaGrande,positive,SIDE TO SIDE 😘 @arianagrande #sidetoside #aria...
4,802001659970744064,#ArianaGrande,positive,Hairspray Live! Previews at the Macy's Thanksg...
...,...,...,...,...
6180,805696468959002624,zac efron,positive,"Abby *talking about the Hamilton soundtrack"": ..."
6181,805699412257181697,zac efron,positive,can we like get zac efron or justin bieber for...
6182,805699615781625856,zac efron,positive,@dansen17 update: Zac Efron kissing a puppy ht...
6183,805701818357579776,zac efron,positive,First Look at Neighbors 2 with Zac Efron Shirt...


In [0]:
# Checking and compare the frequency with what's written in the paper
# # save dataset as csv file
# task_a_train_df.to_csv('coba_train_a.csv', sep='\t')

# assert task_a_train_df[task_a_train_df['sentiment'] == 'positive'].shape[0] == 19902  #SALAH
assert task_a_train_df[task_a_train_df['sentiment'] == 'negative'].shape[0] == 7840
assert task_a_train_df[task_a_train_df['sentiment'] == 'neutral'].shape[0] == 22591

assert task_a_test_df[task_a_test_df['sentiment'] == 'positive'].shape[0] == 2375
assert task_a_test_df[task_a_test_df['sentiment'] == 'negative'].shape[0] == 3972
assert task_a_test_df[task_a_test_df['sentiment'] == 'neutral'].shape[0] == 5937

assert len(task_b_train_df['topic'].unique()) == 373
assert task_b_train_df[task_b_train_df['sentiment'] == 'positive'].shape[0] == 14951
assert task_b_train_df[task_b_train_df['sentiment'] == 'negative'].shape[0] == 4013
assert task_b_train_df[task_b_train_df['sentiment'] == 'neutral'].shape[0] == 1544


assert len(task_b_test_df['topic'].unique()) == 125
assert task_b_test_df[task_b_test_df['sentiment'] == 'positive'].shape[0] == 2463
assert task_b_test_df[task_b_test_df['sentiment'] == 'negative'].shape[0] == 3722

## Tokenization

In [0]:
def tokenize_df(df, tokenizer, input_column_name='text', output_column_name='tokenized'):
  df[output_column_name] = df.apply(
    lambda row: tokenize_text(
        row[input_column_name],
        tokenizer
    ),
    axis=1)
  return df

In [28]:
tokenizer = nltk.tokenize.TweetTokenizer(
    strip_handles = cfg['preprocessing']['tokenization']['remove_twitter_handle'],
    preserve_case = cfg['preprocessing']['tokenization']['preserve_case']
)

task_a_train_df = tokenize_df(task_a_train_df, tokenizer)
task_b_train_df = tokenize_df(task_b_train_df, tokenizer)
task_a_test_df = tokenize_df(task_a_test_df, tokenizer)
task_b_test_df = tokenize_df(task_b_test_df, tokenizer)


# print sample result
task_b_test_df.head(10)

NameError: ignored

# Exploratory Data Analysis (EDA)

## Word Cloud

In [0]:
def show_wordcloud(text, title, max_words=200):
  wc = WordCloud(
      max_words=max_words,
      background_color='white',
      # ranks_only= frequency
  ).generate(text)

  print(wc.words_)

  fig = plt.figure(1, figsize=(12, 12))
  fig.suptitle(title)
  plt.imshow(wc)
  plt.show()

In [0]:
df = task_b_train_df[task_b_train_df['sentiment'] == 'positive']['text'][4]
print(df)
# print(df.shape)
# show_wordcloud(str(df.values), 'judul', max_words=20000)

# print(df.shape)
word_string = "you verse wrote book stand titlea i you you you kampung"
print(type(word_string))
show_wordcloud(word_string, 'judul', max_words=20000)

In [0]:
task_a_train_df['sentiment'].value_counts().plot(kind='bar',figsize=(7,4));
plt.title('Number of tweets');
plt.xlabel('Sentiment');
plt.ylabel('Total tweets');

In [0]:
task_b_train_df[task_b_train_df['text'].str.contains('not')]

In [0]:
positive = task_b_train_df[task_b_train_df['sentiment'] == 'positive']['topic'].value_counts(sort=False).to_frame(name='positive')
negative = task_b_train_df[task_b_train_df['sentiment'] == 'negative']['topic'].value_counts(sort=False).to_frame(name='negative')
neutral = task_b_train_df[task_b_train_df['sentiment'] == 'neutral']['topic'].value_counts(sort=False).to_frame(name='neutral')
# negative = task_b_train_df['topic'][task_b_train_df['sentiment'] == 'negative'].value_counts(sort=False)
# neutral = task_b_train_df['topic'][task_b_train_df['sentiment'] == 'neutral'].value_counts(sort=False)
joined = positive.join(negative).join(neutral)

joined = joined[(joined['positive'] > 0) & (joined['negative']>0)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(joined)

plt.bar(range(joined.shape[0]), joined['positive'], label='positive', color='blue')
plt.bar(range(joined.shape[0]), joined['negative'], label='negative', color='red', bottom=joined['positive'])

In [0]:
# # save dataset as csv file
# task_a_train_df.to_csv('coba_train_a', sep='\t')

# Pytorch

## Build Dataset


In [0]:
from torch.utils.data import DataLoader, Dataset
import csv

class TextDataset(Dataset):
  def __init__(self, dataset_df):
    self.samples = []

    for idx, row in dataset_df.iterrows():
      if idx == 101:
        break
      self.samples.append( (row['text'], row['sentiment']) )

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, idx):
    text, sentiment = self.samples[idx]
    return text, sentiment

  

task_a_trainset = TextDataset(task_a_train_df)
dataloader = DataLoader(
                task_a_trainset,
                batch_size=50,
                num_workers=2
             )

In [0]:
# for i, batch in enumerate(dataloader):
#   for sample in batch:
#     print(batch)
#   print(">>>>>>>")

## Neural Network

three layers:
1. embedding layer (transform one-hot encoding vector into a dense embedding vector)
2. RNN
3. linear layer (output)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim)
    self.fully_connected = nn.Linear(hidden_dim, output_dim)

  def forward(self, text):
    # text dimention is [sentence len, batch size]
    
    # embedded dimention is [sentence len, batch size, embedding_dim]
    embedded = self.embedding(text)

    # output dimention is [sentence len, batch size, hidden dim]
    # hidden dimension is [1, batch size, hidden dim]
    output, hidden = self.rnn(embedded)

    assert torch.equal(output[-1,:,:], hidden.squeeze(0))
    
    return self.fully_connected(hidden.squeze(0))


In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)